In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
å# Load model and tokenizer
model_name = "google/gemma-3-270m"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [9]:
# --- Standard Way (using input_ids) ---
input_text = "Hello, world!"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# The model will look up the embeddings for these IDs
outputs_standard = model(input_ids=input_ids)


In [13]:
embedding_layer = model.get_input_embeddings()
text_embeddings = embedding_layer(input_ids)
outputs_advanced = model(inputs_embeds=text_embeddings)

In [14]:
text_embeddings.shape

torch.Size([1, 5, 640])

In [20]:
embedding_layer = model.get_input_embeddings()
print(embedding_layer.embedding_dim)

640
